<a href="https://colab.research.google.com/github/MarcusL-Dev/atividade1_IA/blob/main/atv1_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import math
import random
import time
import threading

def carregaGrafo(arquivo):
  grafo = {}
  with open(arquivo, 'r') as arq:
    for linha in arq:
      partes = linha.split()
      if partes[0] == 'a':
        no1, no2, peso = int(partes[1]), int(partes[2]), int(partes[3])
        if no1 not in grafo:
          grafo[no1] = []
        grafo[no1].append((no2, peso))
    return grafo

def carregaCoordenadas(arquivo):
  coordenadas = {}
  with open(arquivo, 'r') as arq:
    for linha in arq:
      partes = linha.split()
      if partes[0] == 'v':
        no, latitude, longitude = int(partes[1]), int(partes[3])/1000000, int(partes[2])/1000000
        if no not in coordenadas:
          coordenadas[no] = (latitude, longitude)
  return coordenadas

def converteCoordenadasEmNo(coordenadas, latitude, longitude):
  for no in coordenadas:
    if ( latitude == coordenadas[no][0] ) and ( longitude == coordenadas[no][1] ):
      return no

def distanciaReal(latitudeInicial, longitudeInicial, latitudeFinal, longitudeFinal):
  cateto1 = latitudeFinal - latitudeInicial
  cateto2 = longitudeFinal - longitudeInicial
  distancia = math.sqrt( ( cateto1**2 + cateto2**2 ) )
  return distancia

def haversine(latitudeInicial, longitudeInicial, latitudeFinal, longitudeFinal):
  latitude = math.radians(latitudeFinal) - math.radians(latitudeInicial)
  longitude = math.radians(longitudeFinal) - math.radians(longitudeInicial)

  somaSenos = ( math.sin(latitude/2)**2 ) + ( math.cos(latitudeInicial) * math.cos(latitudeFinal) * ( math.sin(longitude/2)**2 ) )
  if somaSenos < 0:
    somaSenos = somaSenos*-1

  angulo = 2* math.atan2(math.sqrt(somaSenos), math.sqrt(1-somaSenos))

  distancia = angulo*6371.0
  return distancia

def sorteiaNos(coordenadas):
  indiceNo = random.randint(0, len(coordenadas))
  return coordenadas[indiceNo]

def buscaLargura(grafo, noInicial, noFinal, resultado, tempoEvento, resultadoEvent):
  print('Algoritimo Busca em largura')
  print('No inicial: '+str(noInicial)+', no Final: '+str(noFinal))
  print('procurando...')

  visitados = []
  fila = []
  pais = {}
  pai = None

  fila.append(noInicial)

  while len(fila) > 0:
    if tempoEvento.is_set():
      return None
    noAtual = fila.pop(0)

    if pai == None:
      pais[noInicial] = noAtual

    if noAtual == noFinal:
      caminhoInv = []
      caminhoInv.append(noFinal)
      while caminhoInv[-1] != noInicial:
        caminhoInv.append(pais[noAtual])
        noAtual = pais[noAtual]

      caminho = []
      tamanho = len(caminhoInv)
      while tamanho > 0:
        caminho.append(caminhoInv[tamanho-1])
        tamanho = tamanho-1

      resultado.append(caminho)
      resultado.append(len(visitados))
      resultado.append(pais)
      resultado.append(None)
      resultadoEvent.set()
      return resultado

    else:
      if noAtual in grafo:
        for vizinho, peso in grafo[noAtual]:
          if vizinho not in visitados:

            visitados.append(vizinho)
            fila.append(vizinho)
            pai = noAtual
            pais[vizinho] = pai

  resultado.append(None)
  resultadoEvent.set()
  return resultado

def buscaProfundidade(grafo, noInicial, noFinal, resultado, tempoEvento, resultadoEvent):
  print('Algoritimo Busca em profundidade')
  print('No inicial: '+str(noInicial)+', no Final: '+str(noFinal))
  print('procurando...')

  visitados = []
  pilha = []
  pais = {}
  pai = None

  pilha.append(noInicial)
  visitados.append(noInicial)

  while len(pilha) > 0:
    if tempoEvento.is_set():
      return None
    noAtual = pilha.pop()

    if pai == None:
      pais[noInicial] = noAtual

    if noAtual == noFinal:
      caminhoInv = []
      caminhoInv.append(noFinal)
      while caminhoInv[-1] != noInicial:
        caminhoInv.append(pais[noAtual])
        noAtual = pais[noAtual]

      caminho = []
      tamanho = len(caminhoInv)
      while tamanho > 0:
        caminho.append(caminhoInv[tamanho-1])
        tamanho = tamanho-1

      resultado.append(caminho)
      resultado.append(len(visitados))
      resultado.append(pais)
      resultado.append(None)
      resultadoEvent.set()
      return resultado

    else:
      if noAtual in grafo:
        for vizinho, peso in grafo[noAtual]:
          if vizinho not in visitados:
            visitados.append(vizinho)
            pilha.append(vizinho)
            pai = noAtual
            pais[vizinho] = pai

  resultado.append(None)
  resultadoEvent.set()
  return resultado

def buscaUniforme(grafo, noInicial, noFinal, resultado, tempoEvento, resultadoEvent):
  print('Algoritimo busca de custo ubiforme')
  print('No inicial: '+str(noInicial)+', no Final: '+str(noFinal))
  print('procurando...')

  visitados = {}
  filaPrioridade = []
  pais = {}

  pesoTotal = 0
  pai = None

  filaPrioridade.append((0, noInicial))
  visitados[noInicial] = 0

  while len(filaPrioridade) > 0:
    if tempoEvento.is_set():
      return None
    filaPrioridade.sort()

    noAtual = filaPrioridade.pop(0)[1]
    pesoAcumulado = visitados[noAtual]

    if pai == None:
      pais[noAtual] = [noAtual, 0]

    if noAtual == noFinal:
      caminhoInv = []
      caminhoInv.append(noFinal)

      while caminhoInv[-1] != noInicial:
        pesoTotal = pesoTotal + pais[noAtual][1]
        caminhoInv.append(pais[noAtual][0])
        noAtual = pais[noAtual][0]

      caminho = []
      tamanho = len(caminhoInv)
      while tamanho > 0:
        caminho.append(caminhoInv[tamanho-1])
        tamanho = tamanho-1

      resultado.append(caminho)
      resultado.append(len(visitados))
      resultado.append(pais)
      resultado.append(pesoTotal)
      resultadoEvent.set()
      return resultado

    else:
      if noAtual in grafo:
        for vizinho, peso in grafo[noAtual]:
          if vizinho not in visitados:
            visitados[vizinho] = pesoAcumulado+peso
            pai = noAtual
            pais[vizinho] = [pai, peso]

            filaPrioridade.append((pesoAcumulado+peso, vizinho))

  resultado.append(None)
  resultadoEvent.set()
  return resultado

def aEstrela(grafo, noInicial, noFinal, coordenadas, heuristica, resultado, tempoEvento, resultadoEvent):
  if heuristica == 1:
    print('Algoritimo A* euclidiana')
  else:
    print('Algoritimo A* haversine')

  print('No inicial: '+str(noInicial)+', no Final: '+str(noFinal))
  print('procurando...')

  visitados = {}
  filaPrioridade = []
  pais = {}

  pesoTotal = 0
  pai = None

  latitudeFinal, longitudeFinal = coordenadas[noFinal]

  filaPrioridade.append((0, noInicial))
  visitados[noInicial] = 0

  while len(filaPrioridade) > 0:
    if tempoEvento.is_set():
      return None
    filaPrioridade.sort()

    noAtual = filaPrioridade.pop(0)[1]
    pesoAcumulado = visitados[noAtual]

    if pai == None:
      pais[noAtual] = [noAtual, 0]

    if noAtual == noFinal:
      caminhoInv = []
      caminhoInv.append(noFinal)

      while caminhoInv[-1] != noInicial:
        pesoTotal = pesoTotal + pais[noAtual][1]
        caminhoInv.append(pais[noAtual][0])
        noAtual = pais[noAtual][0]

      caminho = []
      tamanho = len(caminhoInv)
      while tamanho > 0:
        caminho.append(caminhoInv[tamanho-1])
        tamanho = tamanho-1

      resultado.append(caminho)
      resultado.append(len(visitados))
      resultado.append(pais)
      resultado.append(pesoTotal)
      resultadoEvent.set()
      return resultado

    else:
      if noAtual in grafo:
        for vizinho, peso in grafo[noAtual]:
          if vizinho not in visitados:

            visitados[vizinho] = pesoAcumulado+peso
            pai = noAtual
            pais[vizinho] = [pai, peso]

            latitudeInicial, longitudeInicial = coordenadas[vizinho]

            if heuristica == 1:
              pesoHeuristico = distanciaReal(latitudeInicial, longitudeInicial, latitudeFinal, longitudeFinal)
            else:
              pesoHeuristico = haversine(latitudeInicial, longitudeInicial, latitudeFinal, longitudeFinal)

            pesoPrioridade = peso + pesoAcumulado + pesoHeuristico
            filaPrioridade.append((pesoPrioridade, vizinho))

  resultado.append(None)
  resultadoEvent.set()
  return resultado

def calcularMemoria(numNosExpandidos):
  memoriaPorNo = 28
  return numNosExpandidos*memoriaPorNo

def calcularFatorRamificacao(pais):
  ramificados = {}
  for pai in pais.copy():
    if isinstance(pais[pai], list):
      if pais[pai][0] not in ramificados:
        ramificados[pais[pai][0]] = 1
      else:
        ramificados[pais[pai][0]] = ramificados[pais[pai][0]]+1
    else:
      if pais[pai] not in ramificados:
        ramificados[pais[pai]] = 1
      else:
        ramificados[pais[pai]] = ramificados[pais[pai]]+1

  totalRamificados = 0
  for ramificado in ramificados:
    totalRamificados = totalRamificados + ramificados[ramificado]

  return totalRamificados/len(ramificados)

def controlador():
  optsGrafo = [1, 2, 3, 4]
  optGrafo = int(input('Qual grafo quer usar? 1-central, 2-eastern, 3-western, 4-newYork: '))
  while(optGrafo not in optsGrafo):
    opt = int(input('Informe uma opcao valida. 1-central, 2-eastern, 3-western, 4-newYork'))
  print('carregando...')

  if optGrafo == 1:
    grafo = carregaGrafo('drive/MyDrive/dimacs/centralUSA/USA-road-d.CTR.gr')
    coordenadas = carregaCoordenadas('drive/MyDrive/dimacs/centralUSA/USA-road-d.CTR.co')
  elif optGrafo == 2:
    grafo = carregaGrafo('drive/MyDrive/dimacs/easternUSA/USA-road-d.E.gr')
    coordenadas = carregaCoordenadas('drive/MyDrive/dimacs/easternUSA/USA-road-d.E.co')
  elif optGrafo == 3:
    grafo = carregaGrafo('drive/MyDrive/dimacs/westernUSA/USA-road-d.W.gr')
    coordenadas = carregaCoordenadas('drive/MyDrive/dimacs/westernUSA/USA-road-d.W.co')
  else:
    grafo = carregaGrafo('drive/MyDrive/dimacs/nyUSA/USA-road-d.NY.gr')
    coordenadas = carregaCoordenadas('drive/MyDrive/dimacs/nyUSA/USA-road-d.NY.co')

  optsEscolhaNos = [1, 2]
  optEscolhaNos = int(input('Qual escolha de nós quer usar? 1-escolha de coordenadas aleatorias, 2-inserir coordenadas manualmente: '))
  while(optEscolhaNos not in optsEscolhaNos):
    optEscolhaNos = int(input('Informe uma opcao valida. 1-escolha de coordenadas aleatorias, 2-inserir coordenadas manualmente: '))

  if optEscolhaNos == 2:
    print('Tome cuidado com as coordenadas invertidas da base dmacs')
    latitudeInicial = int(input('latitude inicial: '))/1000000
    longitudeInicial = int(input('longitude inicial: '))/1000000
    latitudeFinal = int(input('latitude final: '))/1000000
    longitudeFinal = int(input('longitude final: '))/1000000
    numExecucoes = 1
  else:
    numExecucoes = int(input('quantos pares de nos aleatorios? '))
    if numExecucoes < 0:
      numExecucoes = numExecucoes*-1

  algoritimosUsados = [0, 0, 0, 0, 0]

  algoritimosUsados[0] = int(input('quer habilitar o busca em largura? 1-sim, 0-nao '))
  algoritimosUsados[1] = int(input('quer habilitar o busca em profundidade? 1-sim, 0-nao '))
  algoritimosUsados[2] = int(input('quer habilitar o busca de custo uniforme? 1-sim, 0-nao '))
  algoritimosUsados[3] = int(input('quer habilitar o A* euclidiana? 1-sim, 0-nao '))
  algoritimosUsados[4] = int(input('quer habilitar o A* haversine? 1-sim, 0-nao '))

  print()

  for i in range(numExecucoes):
    print(str(i+1)+'° par de nos:\n')
    if optEscolhaNos != 2:
      latitudeInicial, longitudeInicial = sorteiaNos(coordenadas)
      latitudeFinal, longitudeFinal = sorteiaNos(coordenadas)

    noInicial = converteCoordenadasEmNo(coordenadas, latitudeInicial, longitudeInicial)
    noFinal = converteCoordenadasEmNo(coordenadas, latitudeFinal, longitudeFinal)

    count = 0
    for algoritimo in algoritimosUsados:
      if algoritimo == 1:

        tempoLimite = 300
        tempoEvento = threading.Event()
        resultadoEvento = threading.Event()
        resultado = []
        inicio = time.time()

        if count == 4:
          thread = threading.Thread(target=aEstrela, args=(grafo, noInicial, noFinal, coordenadas, 2, resultado, tempoEvento, resultadoEvento)).start()
        elif count == 3:
          thread = threading.Thread(target=aEstrela, args=(grafo, noInicial, noFinal, coordenadas, 1, resultado, tempoEvento, resultadoEvento)).start()
        elif count == 2:
          thread = threading.Thread(target=buscaUniforme, args=(grafo, noInicial, noFinal, resultado, tempoEvento, resultadoEvento)).start()
        elif count == 1:
          thread = threading.Thread(target=buscaProfundidade, args=(grafo, noInicial, noFinal, resultado, tempoEvento, resultadoEvento)).start()
        else:
          thread = threading.Thread(target=buscaLargura, args=(grafo, noInicial, noFinal, resultado, tempoEvento, resultadoEvento)).start()

        fim = time.time()
        tempoGasto = fim - inicio
        while tempoGasto < tempoLimite and not resultadoEvento.is_set():
          fim = time.time()
          tempoGasto = fim - inicio

        if resultadoEvento.is_set():
          if resultado[0] != None:
            caminho = resultado[0]
            numNosExpandidos = resultado[1]
            pais = resultado[2]
            peso = resultado[3]

            print('Caminho: '+str(caminho))

            if peso != None:
              print('Peso do caminho: '+str(peso))

            print('Tempo de execucao: '+str(round(tempoGasto, 2))+' segundos')
            print('Numero de nos expandidos: '+str(numNosExpandidos))
            print('Memoria usada: '+str(calcularMemoria(numNosExpandidos))+' bytes')
            print('Fator de ramifiacao: '+str(round(calcularFatorRamificacao(pais), 2)))
          else:
            print('Nao existe um caminho')
        else:
          tempoEvento.set()
          print('Tempo excedido')
        print('\n')
      count = count + 1

controlador()

Qual grafo quer usar? 1-central, 2-eastern, 3-western, 4-newYork: 4
carregando...
Qual escolha de nós quer usar? 1-escolha de coordenadas aleatorias, 2-inserir coordenadas manualmente: 1
quantos pares de nos aleatorios? 1
quer habilitar o busca em largura? 1-sim, 0-nao 1
quer habilitar o busca em profundidade? 1-sim, 0-nao 1
quer habilitar o busca de custo uniforme? 1-sim, 0-nao 1
quer habilitar o A* euclidiana? 1-sim, 0-nao 1
quer habilitar o A* haversine? 1-sim, 0-nao 1

1° par de nos:

Algoritimo Busca em largura
No inicial: 130529, no Final: 202515
procurando...
Tempo excedido


Algoritimo Busca em profundidade
No inicial: 130529, no Final: 202515
procurando...
Caminho: [130529, 130761, 130763, 130762, 130766, 130771, 48249, 48246, 48191, 48192, 48193, 48194, 48196, 48195, 48254, 48286, 48287, 48289, 48292, 48294, 48293, 48295, 48297, 48204, 48238, 48237, 48203, 48200, 48233, 48232, 48241, 48989, 48992, 48993, 49004, 49005, 49028, 49030, 49037, 49041, 49180, 49193, 49195, 49196, 49

-73092645 41317557 -73228258 41366595